### Import packages

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
from matplotlib.ticker import AutoLocator, AutoMinorLocator, LogLocator
import glob


import os
import sys
from pathlib import Path

# Where am I running?
try:
    # Normal script
    here = Path(__file__).resolve().parent
except NameError:
    # Notebook / REPL
    here = Path.cwd()

phys_const_path = (here / '..' / 'phys_const').resolve()
sys.path.append(str(phys_const_path))

nsm_plots_path = (here / '..' / 'nsm_plots').resolve()
sys.path.append(str(nsm_plots_path))

import phys_const as pc
import plot_functions as pf

### Directory and file names to be analyzed

In [ ]:
direct = '/pscratch/sd/u/uo1999/gw170817_sim/att_convergence_010m/att_1.0e-05_0.65_ye'
finaldir = 'att_1.0e-05_0.65_ye'
file = '/allData.h5'
parfile = '/plt09600_particles'
num_particles_per_energy_bin = 92
num_energy_bins = 13

direct = '/pscratch/sd/u/uo1999/gw170817_sim/att_convergence_010m/att_1.0e-05'
finaldir = 'att_1.0e-05'
file = '/allData.h5'
parfile = '/plt09600_particles'
num_particles_per_energy_bin = 92
num_energy_bins = 13

direct = '/pscratch/sd/u/uo1999/gw170817_1.00Ye/classical_transport/92_ppEb'
finaldir = '92_ppEb'
file = '/allData.h5'
parfile = '/plt00384_particles'
num_particles_per_energy_bin = 92
num_energy_bins = 13
time_s = 0.32/1000 # seconds

direct = '/pscratch/sd/u/uo1999/gw170817_1.00Ye/classical_transport/378_ppEb'
finaldir = '378_ppEb'
file = '/allData.h5'
parfile = '/plt00960_particles'
num_particles_per_energy_bin = 378
num_energy_bins = 13
time_s = 0.32/1000 # seconds

direct = '/pscratch/sd/u/uo1999/gw170817_1.00Ye/classical_transport/1506_ppEb'
finaldir = '1506_ppEb'
file = '/allData.h5'
parfile = '/plt00384_particles'
num_particles_per_energy_bin = 1506
num_energy_bins = 13
time_s = 0.32/1000 # seconds

### Define mesh and black hole and boundary mask

In [ ]:
rho_ye_T_h5py = h5py.File(direct+'/rho_Ye_T.hdf5', 'r')

# number of cells
Nx = np.array(rho_ye_T_h5py['/ncellsx'])
Ny = np.array(rho_ye_T_h5py['/ncellsy'])
Nz = np.array(rho_ye_T_h5py['/ncellsz'])
print(f'Nx = {Nx}, Ny = {Ny}, Nz = {Nz}')

# cell size
dx = ( np.array(rho_ye_T_h5py['/xmax_cm']) - np.array(rho_ye_T_h5py['/xmin_cm']) ) / np.array(rho_ye_T_h5py['/ncellsx']) # cm
dy = ( np.array(rho_ye_T_h5py['/ymax_cm']) - np.array(rho_ye_T_h5py['/ymin_cm']) ) / np.array(rho_ye_T_h5py['/ncellsy']) # cm
dz = ( np.array(rho_ye_T_h5py['/zmax_cm']) - np.array(rho_ye_T_h5py['/zmin_cm']) ) / np.array(rho_ye_T_h5py['/ncellsz']) # cm
cellvolume = dx * dy * dz # cm^3
print(f'dx = {dx} cm, dy = {dy} cm, dz = {dz} cm')

# cell centers
xc = np.linspace(dx / 2, dx * (Nx - 0.5), Nx) # cm
yc = np.linspace(dy / 2, dy * (Ny - 0.5), Ny) # cm
zc = np.linspace(dz / 2, dz * (Nz - 0.5), Nz) # cm

# cell centers mesh
Xc, Yc, Zc = np.meshgrid(xc, yc, zc, indexing='ij') # cm

bh_radius = 5.43e+05 # cm
bh_center_x = 48.0e+5 # cm
bh_center_y = 48.0e+5 # cm
bh_center_z = 16.0e+5 # cm

distance_from_bh = np.sqrt( (Xc - bh_center_x)**2 + (Yc - bh_center_y)**2 + (Zc - bh_center_z)**2 )
mask_bh = (distance_from_bh < bh_radius + 1*np.max([dx,dy,dz]))
mask_x = (Xc < dx) | (Xc > (Nx - 1) * dx)
mask_y = (Yc < dy) | (Yc > (Ny - 1) * dy)
mask_z = (Zc < dz) | (Zc > (Nz - 1) * dz)
mask = mask_bh | mask_x | mask_y | mask_z

rho_ye_T_h5py.close()

### Define arrays that containg the cell for which particles data is saved

In [ ]:
# Get the cell indices
cell_file_names = glob.glob(direct + parfile + '/FFI_cell_*_*_*')
cell_file_names = [file_name.split('/')[-1] for file_name in cell_file_names]
x_cell_ind = np.array([int(file_name.split('_')[2]) for file_name in cell_file_names])
y_cell_ind = np.array([int(file_name.split('_')[3]) for file_name in cell_file_names])
z_cell_ind = np.array([int((file_name.split('_')[4]).split('.')[0]) for file_name in cell_file_names])
cell_indices = np.array(list(zip(x_cell_ind, y_cell_ind, z_cell_ind)))
print('Number of cells:', len(cell_indices))
print(f'shape of cell_indices: {cell_indices.shape}')

# Get the cell indices for ix fix but iy and iz varying all available cells
x_idx_slice = 48
mask_yz_slice = cell_indices[:,0] == x_idx_slice # fixing the x index in this value
cell_indices_yz_slice = cell_indices[mask_yz_slice]
fig, ax = plt.subplots(figsize=(12, 8))
ax.scatter(cell_indices_yz_slice[:,1], cell_indices_yz_slice[:,2], color='b')
ax.set_xlabel('$i_y$')
ax.set_ylabel('$i_z$')
ax.legend()
# ax.set_xlim([-5,100])
# ax.set_ylim([-5,70])
plt.show()
plt.close(fig)

# Get the cell indices for iy fix but ix and iz varying all available cells
y_idx_slice = 48
mask_xz_slice = cell_indices[:,1] == y_idx_slice # fixing the y index in this value
cell_indices_xz_slice = cell_indices[mask_xz_slice]
fig, ax = plt.subplots(figsize=(12, 8))
ax.scatter(cell_indices_xz_slice[:,0], cell_indices_xz_slice[:,2], color='b')
ax.set_xlabel('$i_x$')
ax.set_ylabel('$i_z$')
ax.legend()
# ax.set_xlim([-5,100])
# ax.set_ylim([-5,70])
plt.show()
plt.close(fig)

# Get the cell indices for iz fix but ix and iy varying all available cells
z_idx_slice = 16
mask_xy_slice = cell_indices[:,2] == z_idx_slice # fixing the z index in this value
cell_indices_xy_slice = cell_indices[mask_xy_slice]
fig, ax = plt.subplots(figsize=(12, 8))
ax.scatter(cell_indices_xy_slice[:,0], cell_indices_xy_slice[:,1], color='b')
ax.set_xlabel('$i_x$')
ax.set_ylabel('$i_y$')
ax.legend()
# ax.set_xlim([-5,100])
# ax.set_ylim([-5,70])
plt.show()
plt.close(fig)

# Combine all cell indices from the three slices
# cell_indices_all = np.concatenate((cell_indices_yz_slice, cell_indices_xz_slice, cell_indices_xy_slice), axis=0)
cell_indices_all = np.concatenate((cell_indices_xz_slice, cell_indices_xy_slice), axis=0)
cell_indices_all = np.unique(cell_indices_all, axis=0)

### Reading FFI quantities

In [ ]:
sigma = np.full_like(Xc, np.nan)
GnPos = np.full_like(Xc, np.nan)
GnNeg = np.full_like(Xc, np.nan)

for indexpair in cell_indices_all:
        
        i = indexpair[0]
        j = indexpair[1]
        k = indexpair[2]
        
        FFI_file_name = f'/FFI_cell_{i}_{j}_{k}.h5'
        
        FFI_cell_file = h5py.File(direct + parfile + FFI_file_name, 'r')

        GnNeg[i, j, k] = FFI_cell_file['/GnNeg_inverse_s'][()]
        GnPos[i, j, k] = FFI_cell_file['/GnPos_inverse_s'][()]
        sigma[i, j, k] = FFI_cell_file['/sigma_inverse_s'][()]

        FFI_cell_file.close()

### Aplying mask to plot data

In [ ]:
sigma_FFI = sigma.copy()
GnPos_FFI = GnPos.copy()
GnNeg_FFI = GnNeg.copy()

sigma_FFI[mask] = np.nan
GnPos_FFI[mask] = np.nan
GnNeg_FFI[mask] = np.nan

sigma_FFI[sigma == 0] = np.nan
GnPos_FFI[GnPos == 0] = np.nan
GnNeg_FFI[GnNeg == 0] = np.nan

log10_sigma_FFI = np.log10(sigma_FFI)
log10_GnPos_FFI = np.log10(GnPos_FFI)
log10_GnNeg_FFI = np.log10(GnNeg_FFI)

### Plotting FFI growth rate

In [ ]:
combined_max= 9.937610573617365
combined_min= 5.517461600506053

pf.plot_color_map_with_scattered_points(
    x=Xc[1:-1,y_idx_slice,1:-1]/1.0e5,  # Convert to km
    y=Zc[1:-1,y_idx_slice,1:-1]/1.0e5,  # Convert to km
    z=log10_sigma_FFI[1:-1,y_idx_slice,1:-1],
    bh_r=5.43, # km
    bh_x=48.0, # km
    bh_y=16.0, # km
    min_cb=combined_min,
    max_cb=combined_max,
    x_label=r'$x \, (\mathrm{km})$',
    y_label=r'$z \, (\mathrm{km})$',
    title=fr'$y={Yc[0,y_idx_slice,0]/1.0e5}\,\mathrm{{km}}$, $t = {time_s/1e-3:.2f}\,\mathrm{{ms}}$',
    # cbar_label=r'$\log(\,\sigma_{FFI} \, [\mathrm{1/s}]\,) \, , \, \sigma_{FFI} = \sqrt{AB}$',
    cbar_label=r'$\log(\,\sigma_{FFI} \, [\mathrm{1/s}]\,)$',
    colormap='plasma',
    filename='plots/'+finaldir+f'_{time_s/1e-3:.2f}ms_sigmaFFI_xz.png',
    # x_scat=[05.5, 10.5, 15.5, 20.5, 25.5, 30.5, 35.5, 40.5],
    # y_scat=[15.5, 15.5, 15.5, 15.5, 15.5, 15.5, 15.5, 15.5],
    # x_scat=[10.5, 10.5, 10.5, 10.5, 10.5, 10.5, 10.5, 10.5, 10.5, 10.5],
    # y_scat=[15.5, 20.5, 25.5, 30.5, 35.5, 40.5, 45.5, 50.5, 55.5, 60.5],
    # x_scat=[48.5, 48.5, 48.5, 48.5, 48.5, 48.5, 48.5, 48.5],
    # y_scat=[25.5, 30.5, 35.5, 40.5, 45.5, 50.5, 55.5, 60.5],
    # x_scat=[15.5],
    # y_scat=[15.5],
    x_scat=None,
    y_scat=None,
    size_scat=50,
    marker_scat='x',
    color_scat='black',
    # x_scat=None, 
    # y_scat=None, 
    # size_scat=None, 
    # marker_scat=None, 
    # color_scat=None,
    doshow=True,
    dosave=True 
)

pf.plot_color_map_with_scattered_points(
    x=Xc[1:-1,1:-1,z_idx_slice]/1.0e5,  # Convert to km
    y=Yc[1:-1,1:-1,z_idx_slice]/1.0e5,  # Convert to km
    z=log10_sigma_FFI[1:-1,1:-1,z_idx_slice],
    bh_r=5.43, # km
    bh_x=48.0, # km
    bh_y=48.0, # km
    min_cb=combined_min,
    max_cb=combined_max,
    x_label=r'$x \, (\mathrm{km})$',
    y_label=r'$y \, (\mathrm{km})$',
    title=fr'$z={Zc[0,0,z_idx_slice]/1.0e5}\,\mathrm{{km}}$, $t = {time_s/1e-3:.2f}\,\mathrm{{ms}}$',
    # cbar_label=r'$\log(\,\sigma_{FFI} \, [\mathrm{1/s}]\,) \, , \, \sigma_{FFI} = \sqrt{AB}$',
    cbar_label=r'$\log(\,\sigma_{FFI} \, [\mathrm{1/s}]\,)$',
    colormap='plasma',
    filename='plots/'+finaldir+f'_{time_s/1e-3:.2f}ms_sigmaFFI_xy.png',
    # x_scat=[05.5, 10.5, 15.5, 20.5, 25.5, 30.5, 35.5, 40.5], 
    # y_scat=[48.5, 48.5, 48.5, 48.5, 48.5, 48.5, 48.5, 48.5], 
    # x_scat=[10.5],
    # y_scat=[48.5],
    # x_scat=[15.5],
    # y_scat=[48.5], 
    x_scat=None,
    y_scat=None,
    size_scat=50, 
    marker_scat='x', 
    color_scat='black',
    # x_scat=None, 
    # y_scat=None, 
    # size_scat=None, 
    # marker_scat=None, 
    # color_scat=None,
    doshow=True,
    dosave=True
)

with h5py.File(f'plots/{finaldir}_{time_s/1e-3:.2f}ms_sigmaFFI_slices.h5', 'w') as f:

    f.create_dataset('xz_x_km', data=Xc[1:-1, y_idx_slice, 1:-1]/1.0e5)
    f.create_dataset('xz_y_km', data=Zc[1:-1, y_idx_slice, 1:-1]/1.0e5)
    f.create_dataset('xz_z_invsec', data=log10_sigma_FFI[1:-1, y_idx_slice, 1:-1])

    f.create_dataset('xy_x_km', data=Xc[1:-1, 1:-1, z_idx_slice]/1.0e5)
    f.create_dataset('xy_y_km', data=Yc[1:-1, 1:-1, z_idx_slice]/1.0e5)
    f.create_dataset('xy_z_invsec', data=log10_sigma_FFI[1:-1, 1:-1, z_idx_slice])

# Compute individual slice extrema
xz_max = np.nanmax(log10_sigma_FFI[:, y_idx_slice, :])
xz_min = np.nanmin(log10_sigma_FFI[:, y_idx_slice, :])

xy_max = np.nanmax(log10_sigma_FFI[:, :, z_idx_slice])
xy_min = np.nanmin(log10_sigma_FFI[:, :, z_idx_slice])

# Combined extrema
combined_max = max(xz_max, xy_max)
combined_min = min(xz_min, xy_min)

print("combined_max=", combined_max)
print("combined_min=", combined_min)

### Plotting positive G_ELN integral

In [ ]:
pf.plot_color_map(
    x=Xc[:,y_idx_slice,:]/1.0e5,  # Convert to km
    y=Zc[:,y_idx_slice,:]/1.0e5,  # Convert to km
    z=log10_GnPos_FFI[:,y_idx_slice,:],
    min_cb=None,
    max_cb=None,
    x_label=r'$x \, (\mathrm{km})$',
    y_label=r'$z \, (\mathrm{km})$',
    title=fr'$y={Yc[0,y_idx_slice,0]/1.0e5}\,\mathrm{{km}}$, $t = {time_s/1e-3:.2f}\,\mathrm{{ms}}$',
    cbar_label='$\log(\,B \, [\mathrm{1/s}]\,) \,,\, B=\int \,G(\hat{n})_{>0}\,d\Omega$',
    colormap='viridis',
    filename='plots/'+finaldir+f'_{time_s/1e-3:.2f}ms_posGN_xz.png',
    doshow=True,
    dosave=True
)

pf.plot_color_map(
    x=Xc[:,:,z_idx_slice]/1.0e5,  # Convert to km
    y=Yc[:,:,z_idx_slice]/1.0e5,  # Convert to km
    z=log10_GnPos_FFI[:,:,z_idx_slice],
    min_cb=None,
    max_cb=None,
    x_label=r'$x \, (\mathrm{km})$',
    y_label=r'$y \, (\mathrm{km})$',
    title=fr'$z={Zc[0,0,z_idx_slice]/1.0e5}\,\mathrm{{km}}$, $t = {time_s/1e-3:.2f}\,\mathrm{{ms}}$',
    cbar_label='$\log(\,B \, [\mathrm{1/s}]\,) \,,\, B=\int \,G(\hat{n})_{>0}\,d\Omega$',
    colormap='viridis',
    # colormap='Oranges',
    filename='plots/'+finaldir+f'_{time_s/1e-3:.2f}ms_posGN_xy.png',
    doshow=True,
    dosave=True
)

### Plotting negative G_ELN integral

In [ ]:
pf.plot_color_map(
    x=Xc[:,y_idx_slice,:]/1.0e5,  # Convert to km
    y=Zc[:,y_idx_slice,:]/1.0e5,  # Convert to km
    z=log10_GnNeg_FFI[:,y_idx_slice,:],
    min_cb=None,
    max_cb=None,
    x_label=r'$x \, (\mathrm{km})$',
    y_label=r'$z \, (\mathrm{km})$',
    title=fr'$y={Yc[0,y_idx_slice,0]/1.0e5}\,\mathrm{{km}}$, $t = {time_s/1e-3:.2f}\,\mathrm{{ms}}$',
    cbar_label='$\log(\,A \, [\mathrm{1/s}]\,) \,,\, A=\int \,G(\hat{n})_{<0}\,d\Omega$',
    colormap='viridis',
    filename='plots/'+finaldir+f'_{time_s/1e-3:.2f}ms_negGN_xz.png',
    doshow=True,
    dosave=True
)

pf.plot_color_map(
    x=Xc[:,:,z_idx_slice]/1.0e5,  # Convert to km
    y=Yc[:,:,z_idx_slice]/1.0e5,  # Convert to km
    z=log10_GnNeg_FFI[:,:,z_idx_slice],
    min_cb=None,
    max_cb=None,
    x_label=r'$x \, (\mathrm{km})$',
    y_label=r'$y \, (\mathrm{km})$',
    title=fr'$z={Zc[0,0,z_idx_slice]/1.0e5}\,\mathrm{{km}}$, $t = {time_s/1e-3:.2f}\,\mathrm{{ms}}$',
    cbar_label='$\log(\,A \, [\mathrm{1/s}]\,) \,,\, A=\int \,G(\hat{n})_{<0}\,d\Omega$',
    colormap='viridis',
    # colormap='Oranges',
    filename='plots/'+finaldir+f'_{time_s/1e-3:.2f}ms_negGN_xy.png',
    doshow=True,
    dosave=True
)